In [ ]:
#Imports
import MDAnalysis as mda
from MDAnalysis.lib import transformations, mdamath
import numpy as np

In [ ]:
#Create the Universe and select the backbone of the protein
protein = mda.Universe("protein.gro")
bbs = protein.select_atoms("name BB")

In [ ]:
#Vector (Valine:387 Tyrosine:45) to align protein with membrane 
vector = bbs[382].position - bbs[40].position

In [ ]:
#Rotate NDH-2
residues_FAD = protein.select_atoms("resname FAD")
FAD_cog = residues_FAD.center_of_geometry()
angle = np.degrees(mdamath.angle(vector, [0,0,-1]))
ax = transformations.rotaxis(vector, [0,0,-1])
protein.atoms.rotateby(angle, ax, point=FAD_cog)

In [ ]:
#Discover the protein size and adjust it to fit into membrane's box
min_z_protein = min(protein.atoms.positions[:,2])
max_z_protein = max(protein.atoms.positions[:,2])

lenprotein = max_z_protein - min_z_protein
lenprotein = round(lenprotein)+3
z_dimension = 30+40+10+lenprotein
z_dimension

In [ ]:
#Set up the new dimensions of protein's box and the protein itself to be right in the middle of the box (so that, then we just need to shift the protein to the top of the membrane using insane.py)
#Discover the membrane dimensions, so that we can know the final dimensions of our box to simulate
membrane = mda.Universe("membrane.gro")
protein.dimensions = [membrane.dimensions[0], membrane.dimensions[1], z_dimension + membrane.dimensions[2], 90, 90, 90]
positions_z_protein = protein.atoms.positions[:,2]
positions_z_protein += (z_dimension)/2+30
protein.atoms.positions[:,2] = positions_z_protein
protein.atoms.write("protein_newbox.gro")

In [ ]:
#We needed to shuffle the waters, in the new box, after solvation - protein_solvated.gro
protein_solvated = mda.Universe("protein_solvated.gro")
W = protein_solvated.select_atoms("name W")
water = W.positions
np.random.shuffle(water)
W.positions = water

In [ ]:
#We, then, added the protein into the membrane using insane.py